In [1]:
# IMPORT SECTION
import pandas as pd
from util_data_exploration import read_CBS_excel
import geopandas as gpd
import difflib  # for comparing deltas and similar-looking matching
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
from scipy.optimize import curve_fit
import os

In [2]:
%%capture
'''
read excel files from CBS data, mostly down to buurt_level:
n_addresses : number of addresses (in lieu of households)
surface_areas : total area within buurt
median_prices : median house prices (probably depreciated)

'''
# use 2022 number of addresses (should not affect much)

n_addresses = read_CBS_excel("data/Bewoonde adressen - 2022 - Gebieden.xlsx",index_name='wijk',convert_to_Int64=True)
# use 2018 data for highest_percentile(
highest20_percentile = read_CBS_excel('data/20% huishoudens met hoogste inkomen - Gebieden.xlsx', index_name='wijk').loc[:,'2018'].rename('high20')
lowest40_percentile = read_CBS_excel('data/40% huishoudens met laagste inkomen - Gebieden.xlsx', index_name='wijk').loc[:,'2018'].rename('low40')
companies = read_CBS_excel("data/Bedrijfsvestigingen naar SBI 1 niveau - Gebieden.xlsx", index_name='wijk',convert_to_Int64=True).loc[:, '2018'].rename('companies')
middle40_percentile = pd.Series(data=(1 - lowest40_percentile - highest20_percentile), name='mid40')

In [3]:
"""
Simple method: drop the
only preserve relevant dataframes:
(transactions shouldn't be the constraint now)


"""
# get the number of households per bracket (column) per area (row)
households_p_bracket = pd.concat([lowest40_percentile, middle40_percentile, highest20_percentile], axis=1).mul(n_addresses.values, axis='index').astype(float).round(decimals=0).astype('Int64')
households_p_bracket.columns = ['low40_hholds', 'mid40_hholds', 'high20_hholds']

In [4]:
# save into input list
households_p_bracket.to_pickle('data_model_inputs/households_brackets_per_wijk.pickletable')
companies.to_pickle('data_model_inputs/companies_per_wijk.pickletable')

In [5]:

# merge into big descriptive dataframe
descriptive = pd.concat(
    [lowest40_percentile, middle40_percentile, highest20_percentile, households_p_bracket, n_addresses, companies], axis=1)
#.dropna(how='any',                                      subset=['low40','high20'])  # since mid40 is derived from low40 and high20, this is fine.


In [6]:
# experiment to see how many classes of agents required


In [7]:
"""
Read vector data from rdam vector maps
"""
## maybe some visualisations to help
overwrite = True  # command to force overwrite
if 'buurten' not in locals() or overwrite:
    # open files for all buurten in NL
    buurten = gpd.read_file("data/WijkBuurtkaart_2020_v2/buurt_2020_v2.shp")
    # sort for Rdam
    rdam_buurten = buurten[buurten.GM_CODE == 'GM0599'].dropna(how='any')  # corresponds to Rdam, buurt only
    # rdam_buurt_2_wijk = rdam_buurten.copy()
    # extrarct only the geometry
    rdam_buurten = rdam_buurten[['BU_NAAM', 'WK_CODE', 'geometry']].set_index('BU_NAAM')
# rdam_buurten.plot()
# activate below to save the geometries for Rdam buurten
# rdam_buurten.to_file('data/Rdam_buurten_shape.shp')

# attempt to match rdam buurten index to descriptive dataframe
descriptive.index = descriptive.index.map(lambda x: difflib.get_close_matches(x, rdam_buurten.index)[0])
descriptive = gpd.GeoDataFrame(descriptive, geometry=rdam_buurten.loc[descriptive.index].geometry)
# descriptive = rdam_buurten.join(descriptive.astype(dtype='float64'),how='left')

IndexError: list index out of range

In [ ]:
overwrite2 = True

if 'wijken' not in locals() or overwrite2:
    wijken = gpd.read_file('data/WijkBuurtkaart_2020_v2/wijk_2020_v2.shp')
    # filter for Rdam
    wijken = wijken[wijken.GM_CODE == 'GM0599'].dropna(how='any')
    # preserve only code, name and geometry
    wijken = wijken.loc[:, ['WK_CODE', 'WK_NAAM', 'geometry']]

    wijken.to_file('data/wijk_geometries.shp')

# TODO:use groupby and aggregation to get aggregation. Prolly need to have some mapping from the buurt data to the wijk data
# Create mapping from wijk code in buurt data to wijk name in wijk data
rdam_buurten['WK_NAAM'] = rdam_buurten.WK_CODE.map(wijken.set_index('WK_CODE').WK_NAAM)

In [ ]:
columns_to_plot = descriptive.columns[:-2].tolist()  # up to 'transactions'
# make figures with titles, and no axes
for idx, entry in enumerate(columns_to_plot):
    fig, ax = plt.subplots()
    descriptive.plot(column=entry, ax=ax, missing_kwds={'color': 'lightgrey'},
                     cmap='viridis')
    # ax.set_axis_off() # turn on for pretty plotting
    ax.set_title(f"{entry} within Rotterdam buurten")
    # fig.show()
    fig.savefig(f'data/choropleths_{entry}.jpg', dpi=800)
# fig.savefig('data/choropleths.jpg',dpi=600 )
# TODO: finish plots for choropleths with titling

In [13]:
# sum the shit
total_households = households_p_bracket.dropna().to_numpy().sum()
# 3033443 households total
# agent-ratio of 1:50 leads to ~6000

In [ ]:
"""
Get income distribution  of neighbourhoods
-> Used to translate to income distribution of agents
Also get number of agents per wijk/buurt
Could integrate number of agents per

Income table derived from onderzoek010 for proportion of brackets
Median income derived from CBS for deciles
Therefore, get distribution of agents (function?)

"""

